In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.
!pip install clean-text
!pip install unicode
# !pip install keras==2.15.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=0223548bdb25e799ce902bbd67d1aca1a2f166851e765cdbd400528ff496aa31
  Stored in directory: /root/.cache/pip/wheels/31/8a/8

In [5]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

import keras_nlp
import keras

# améliore la rapidité d'entrainement
keras.mixed_precision.set_global_policy("mixed_float16")

In [6]:
import pandas as pd

df = pd.concat([
    pd.read_csv('/content/drive/MyDrive/Corona_NLP_test.csv')[['OriginalTweet']],
    pd.read_csv('/content/drive/MyDrive/Corona_NLP_train.csv', encoding='latin-1', on_bad_lines='skip')[['OriginalTweet']]

])
df.reset_index(inplace=True, drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44955 entries, 0 to 44954
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  44955 non-null  object
dtypes: object(1)
memory usage: 351.3+ KB


# **Pré-traitement**

In [7]:
print(df['OriginalTweet'].isnull().sum())
df.dropna(subset=['OriginalTweet'], how='any', inplace=True)
df.reset_index(inplace=True, drop=True)

unique_elements = not df['OriginalTweet'].duplicated().any()

if unique_elements:
    print("Tous les éléments sont uniques.")
else:
    print("Certains éléments ne sont pas uniques.")

0
Tous les éléments sont uniques.


In [8]:
import cleantext
import re
# corrige les caractères unicode mal formaté
# converti les caracteres non ASCII en leur équivalent exemple : é devient e

df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: cleantext.clean(x,
                                                                         fix_unicode=True,
                                                                         to_ascii=True,
                                                                         lower=True,
                                                                         no_emails=True,
                                                                         no_urls=True,
                                                                         ))
df['OriginalTweet'] = df['OriginalTweet'].str.replace('<url>', '')
# traitement des character speciaux
df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: re.sub(r'\W', ' ', x))
# on enleve les espaces en trop
df['OriginalTweet'] = df['OriginalTweet'].str.replace(r'\s{2,}', ' ', regex=True)


In [ ]:
df.info()

In [40]:
# initialisation pour l'utilisation du modèle GPT-2
import pandas as pd
# from tensorflow.keras.preprocessing.text import Tokenizer
from keras_nlp.models import GPT2CausalLMPreprocessor, GPT2CausalLM

preprocessor = GPT2CausalLMPreprocessor.from_preset(
    "gpt2_large_en",
    sequence_length=128,
)

gpt2_lm = GPT2CausalLM.from_preset(
    "gpt2_large_en",
    preprocessor=preprocessor
)

gpt2_lm.summary()


Preprocessor: "gpt2_causal_lm_preprocessor_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                     │                                              50,257 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gpt2_causal_lm_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 1280)        │     774,030,080 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      64,328,960 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 774,030,080 (2.88 GB)

 Trainable params: 774,030,080 (2.88 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(df)

In [12]:
text_df = list(df['OriginalTweet'].astype(str).values)
print(text_df[:10])

['trending new yorkers encounter empty supermarket shelves pictured wegmans in brooklyn sold out online grocers foodkick maxdelivery as coronavirus fearing shoppers stock up ', 'when i couldn t find hand sanitizer at fred meyer i turned to amazon but 114 97 for a 2 pack of purell check out how coronavirus concerns are driving up prices ', 'find out how you can protect yourself and loved ones from coronavirus ', ' panic buying hits newyork city as anxious shoppers stock up on food medical supplies after healthcare worker in her 30s becomes bigapple 1st confirmed coronavirus patient or a bloomberg staged event qanon qanon2018 qanon2020 election2020 cdc ', ' toiletpaper dunnypaper coronavirus coronavirusaustralia coronavirusupdate covid_19 9news corvid19 7newsmelb dunnypapergate costco one week everyone buying baby milk powder the next everyone buying up toilet paper ', 'do you remember the last time you paid 2 99 a gallon for regular gas in los angeles prices at the pump are going down a

In [41]:
import tensorflow as tf


nb_epochs = 5
with tf.device('/device:GPU:0'):

    learning_rate = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=len(text_df) * nb_epochs,
    decay_rate=0.96,
    staircase=True
    )

    # l'otpmisation PolynomialDecay peut être utilisé à la place de Exponential
    # Decay (la précision n'est pas mauvaise)

    # initial_learning_rate peut être mis à 1e-3
    # learning_rate = keras.optimizers.schedules.PolynomialDecay(
    #     initial_learning_rate=5e-5,
    #     decay_steps=len(text_df) * nb_epochs,
    #     end_learning_rate=0.0,
    # )

    # on va crée un callbacks dans le cas ou l'entrainement devient régréssant
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=nb_epochs, restore_best_weights=True)

    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    gpt2_lm.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        weighted_metrics=["accuracy"],
    )

    gpt2_lm.fit(
        x=text_df,
        epochs=nb_epochs,
        # batch_size = 32,
        callbacks=[early_stopping]
    )

Epoch 1/5
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 1236s 568ms/step - accuracy: 0.3385 - loss: 1.2210
Epoch 2/5
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 425s 302ms/step - accuracy: 0.4055 - loss: 1.0146
Epoch 3/5
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 430s 305ms/step - accuracy: 0.4726 - loss: 0.8486
Epoch 4/5
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 430s 305ms/step - accuracy: 0.5486 - loss: 0.6875
Epoch 5/5
1405/1405 ━━━━━━━━━━━━━━━━━━━━ 431s 306ms/step - accuracy: 0.6267 - loss: 0.5447


## Sauvegarde du modèle

On sauvegarde le model pour éviter de relancer toutes les cellules.

In [44]:
from tensorflow.keras.models import load_model, save_model

save_model(gpt2_lm, '/content/drive/MyDrive/my_model.keras')

# Réutilisation d'un modèle sauvegardé

In [ ]:
gpt2_lm = load_model('/content/drive/MyDrive/my_model.keras')

In [43]:
# model gpt-2 de base
for i in range(5):
# Ne pas laisser d'espace avant le " ça un impacte énorme !
  seed_text = "The symptoms of the covid"
  # on peu rajouter le paramètre max_length si on veut
  generated_text = gpt2_lm.generate(seed_text, max_length=100)
  print("\nGPT-2 output:")
  print(generated_text)
  print('___________________________________________________________')


GPT-2 output:
The symptoms of the covid 19 virus can be so severe that a healthcare worker may be too scared to come into work to make money the government needs to step in and save these workers now 
___________________________________________________________

GPT-2 output:
The symptoms of the covid19 coronavirus vary from person to person i have observed that when i have left my house to go to the supermarket the streets have been full of people i have been told that it has been like this every day this week
___________________________________________________________

GPT-2 output:
The symptoms of the covid2019 covid_19 pandemic vary from person to person and may include fever coughing chest congestion shortness of breath muscle aches and pains in various parts of your body learn more about what to look for in your next health exam 
___________________________________________________________

GPT-2 output:
The symptoms of the covid2019 coronavirus vary from person to person but the 